# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [16]:
# Write your code below.

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [18]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA", "data/price_data")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

parquet_files

['../../05_src/data/prices\\ABCB\\ABCB_1994\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1994\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1995\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1995\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1996\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1996\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1997\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1997\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1998\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1998\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1999\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_1999\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2000\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2000\\part.1.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2001\\part.0.parquet',
 '../../05_src/data/prices\\ABCB\\ABCB_2001\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [19]:
# Write your code below.

# Read parquet files into a Dask DataFrame and set the index to "ticker"
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# Add lags for variables Close and Adj_Close
dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

# Add returns based on Close:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

# Add range and assign result to dd_feat:
dd_feat = dd_rets.assign(hi_lo_range = lambda x:
    x['High'] - x['Low'])

dd_feat

C:\Users\Duong\AppData\Local\Temp\ipykernel_23264\2543178867.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=89,,,,,,,,,,,,,
ABCB,datetime64[ns],float64,float64,float64,float64,float64,int64,object,int32,float64,float64,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.

# Convert the Dask DataFrame to a Pandas DataFrame
pd_feat = dd_feat.compute()

# Sort data by ticker and date
pd_feat = pd_feat.sort_values(['ticker', 'Date'], ascending=[True, False])

# Add a moving average of the Returns using a window of 10 days
pd_mov_ave = pd_feat.assign(moving_avg = pd_feat.groupby('ticker')['Returns'].rolling(10).mean().reset_index(level=0, drop = True))

pd_mov_ave


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,moving_avg
ticker,,,,,,,,,,,,,,
ABCB,2020-04-01,22.430000,23.090000,21.750000,22.540001,22.540001,433900.0,ABCB.csv,2020,23.760000,23.760000,-0.051347,1.340000,NaN
ABCB,2020-03-31,23.190001,24.360001,23.129999,23.760000,23.760000,551400.0,ABCB.csv,2020,23.350000,23.350000,0.017559,1.230001,NaN
ABCB,2020-03-30,22.540001,23.469999,21.450001,23.350000,23.350000,462300.0,ABCB.csv,2020,22.520000,22.370001,0.036856,2.019999,NaN
ABCB,2020-03-27,21.860001,23.350000,21.400000,22.520000,22.370001,466600.0,ABCB.csv,2020,22.959999,22.807068,-0.019164,1.950001,NaN
ABCB,2020-03-26,21.950001,23.420000,21.459999,22.959999,22.807068,656400.0,ABCB.csv,2020,21.559999,21.416393,0.064935,1.960001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,1989-11-15,7.533333,7.733333,7.333333,7.533333,7.382380,140000.0,ZIXI.csv,1989,7.533333,7.382380,0.000000,0.400000,0.000113
ZIXI,1989-11-14,7.466667,7.533333,7.333333,7.533333,7.382380,180100.0,ZIXI.csv,1989,7.466667,7.317050,0.008929,0.200000,0.001006
ZIXI,1989-11-13,7.733333,7.800000,7.333333,7.466667,7.317050,293700.0,ZIXI.csv,1989,7.733333,7.578372,-0.034483,0.466667,-0.003335


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

> It was not necessary to convert to pandas to calculate the moving average return since dask also supports rolling operations. 

+ Would it have been better to do it in Dask? Why?

> It would not have been better to do it in Dask for this case. The reason is that it is easier to use pandas for small to medium datasets and more data treatment would be required in Dask to make sure that the data is partitioned correctly for rolling operations.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.